In [105]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [106]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_timestamp.csv')
df = pd.read_csv(csv_file_path) 
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,wday,retry,time,time_class,time_class_mean,time_class_sum,time_class_std
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,1,0.0,3.0,0,0.272458,90967,0.445225
1,0,A060001002,A060000001,1,1585009034,7225,2020,3,24,0,1,0.0,8.0,1,0.709531,281285,0.453979
2,0,A060001003,A060000001,1,1585009042,7225,2020,3,24,0,1,0.0,7.0,1,0.709531,281285,0.453979
3,0,A060001004,A060000001,1,1585009049,7225,2020,3,24,0,1,0.0,7.0,1,0.709531,281285,0.453979
4,0,A060001005,A060000001,1,1585009056,7225,2020,3,24,0,1,0.0,11.0,1,0.709531,281285,0.453979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,2020,6,5,6,4,0.0,27.0,2,0.644391,384028,0.478698
2266582,7441,A040165001,A040000165,1,1597971999,8836,2020,8,21,1,4,0.0,11.0,1,0.709531,281285,0.453979
2266583,7441,A040165002,A040000165,1,1597972010,8836,2020,8,21,1,4,0.0,46.0,4,0.779179,292253,0.414800
2266584,7441,A040165003,A040000165,1,1597972056,8836,2020,8,21,1,4,0.0,73.0,4,0.779179,292253,0.414800


In [107]:
def testid_fe(df):
    ## 시험지의 평균 정답률, 정답 개수, 표준편차
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum','std'])
    correct_t.columns = ["test_ans_mean", 'test_ans_sum','test_ans_std']
    df = pd.merge(df, correct_t, on=['testId'], how="left")

    ## 시험지의 평균 풀이 시간, 표준편차
    time_t = df.groupby(['testId'])['time'].agg(['mean', 'std'])
    time_t.columns = ["test_time_mean", 'test_time_std']
    df = pd.merge(df, time_t, on=['testId'], how="left")


    ## 시험지 대분류(test_type) 생성
    df['test_type'] = df['testId'].apply(lambda x:x[2])

    ## 시험지 대분류별 정답률, 정답 개수, 표준편차
    correct_type = df.groupby(['test_type'])['answerCode'].agg(['mean', 'sum','std'])
    correct_type.columns = ["t_type_ans_mean", 't_type_ans_sum','t_type_ans_std']
    df = pd.merge(df, correct_type, on=['test_type'], how="left")

    ## 시험지 대분류별 풀이시간, 표준편차
    time_type = df.groupby(['test_type'])['time'].agg(['mean', 'std'])
    time_type.columns = ["t_type_time_mean", 't_type_time_std']
    df = pd.merge(df, time_type, on=['test_type'], how="left")

    ## 시험지 난이도
    df['test_lvl'] = df['test_time_mean'] / df['test_ans_mean']
    # 범주화
    cat_num = 10
    df['test_lvl_cat'] = pd.qcut(df['test_lvl'],cat_num,labels=[i for i in range(cat_num)])

    ## 시험지 대분류별 난이도
    # 난이도를 나타내는 척도는 임의로 풀이시간 평균 / 평균 정답률로 설정.
    # 풀이시간이 길어지면 난이도가 높아지고, 정답률이 낮아지면 난이도가 높아진다.
    df['test_type_lvl'] = df['t_type_time_mean'] / df['t_type_ans_mean']

    # 범주화
    type_cat_num = 3
    df['test_type_lvl_cat'] = pd.qcut(df['test_type_lvl'],type_cat_num,labels=[i for i in range(type_cat_num)])

    ## 시험지 노출 횟수
    df['test_total_answer'] = df.groupby('testId')['answerCode'].cumcount()

    return df

In [108]:
df = testid_fe(df)

In [104]:
df.to_csv('/opt/ml/input/data/train_data_test_fe.csv',index=False)